Steps to load:<br>
Project -> Upgrade (if you haven't already)<br>
Run --> Run all cells<br>
Wait a few seconds and scroll down

In [1]:
from collections import OrderedDict 
from ipywidgets import Output, Tab, Textarea, Layout, HTML
import pandas as pd
from plotly.figure_factory import create_dendrogram
from scipy.cluster.hierarchy import dendrogram, linkage
import numpy as np

import bql

from utils_gui import AppTitle, ComputeButton, DatePickerAndLabel, DropdownAndLabel, HBox, TextBoxAndLabel, VBox

bq = bql.Service()

In [2]:
def get_returns_for_pairs(bq, pairs, start_date, end_date, freq):
    univ = bq.univ.list(pairs)
    item = bq.data.px_last(start=start_date, END=end_date, PER=freq)
    req = bql.Request(univ,{'Returns':item})
    res = bq.execute(req)
    df = res[0].df()
    prices = df.reset_index().pivot(index='DATE', columns='ID', values='Returns')
    returns = prices.pct_change().iloc[1:]
    returns = returns.rename(columns={s: s[:3] for s in pairs})
    return returns

In [3]:
def get_returns_for_eq_index_members(bq, index, start_date, end_date, freq, ccy):
    univ = bq.univ.members(index)
    item = bq.data.day_to_day_tot_return_gross_dvds(fill='prev', start=start_date, end=end_date, per=freq, currency=ccy).dropna(remove_id=True)
    req = bql.Request(univ,{'Returns':item})
    res = bq.execute(req)
    df = res[0].df()
    returns = df.reset_index().pivot(index='DATE', columns='ID', values='Returns')
    names = bq.execute(bql.Request(univ, {'Name': bq.data.name()}))[0].df()['Name']
    sectors = bq.execute(bql.Request(univ, {'Sector': bq.data.gics_sector_name().dropna(remove_id=True)}))[0].df()['Sector']
    sector_initials = sectors.map(lambda string: "".join([piece[0] for piece in string.split()]))
    returns = returns.rename(columns={k: sector_initials[k] + " - " + names[k] for k in sector_initials.keys()})
    return returns

In [4]:
def get_returns_for_custom_list(bq, lst, start_date, end_date, freq, ccy):
    univ = bq.univ.list(lst)
    item = bq.data.px_last(start=start_date, END=end_date, PER=freq,fill='prev',currency=ccy).dropna(remove_id=True)
    req1 = bql.Request(univ,{'Returns':item})
    res1 = bq.execute(req1)
    df = res1[0].df()
    req2 = bql.Request(univ,{'Name':bq.data.name()})
    res2 = bq.execute(req2)
    names_map = res2[0].df().T.to_dict('records')
    prices = df.reset_index().pivot(index='DATE', columns='ID', values='Returns')
    returns = prices.pct_change().iloc[1:]
    returns = returns.rename(columns=names_map[0])
    return returns

In [5]:
def get_dendo_from_returns(returns, output, orientation):
    if orientation == 'Vertical':
        orientation_ = 'left'
        width = 1500
        height = len(returns.columns) * 15
    else:
        orientation_ = 'bottom'
        width = 1500
        height = 500
        
    correlation = returns.corr()
    correlation = correlation.dropna(how='all')
    correlation = correlation.dropna(axis=1,how='all')
    correlation = correlation.replace(np.nan,0)
    output.clear_output()
    with output:
        linkfun = lambda x: linkage(x, method='average', metric='euclidean', optimal_ordering=True)
        fig = create_dendrogram(correlation,orientation=orientation_, linkagefun=linkfun, labels=correlation.columns)
        fig.update_layout(
                margin=dict(l=10, r=10, t=10, b=30),
                height=height,
                width=width,
                template='plotly_dark'
        )
        fig.show()
    return None

In [6]:
def compute_fx_dendo(bq, fx, start_date, end_date, freq, output,orientation):
    gui_output = gui_fx.children[-1]
    gui_fx.children = list(gui_fx.children[:-1])+[spinner]
    returns = get_returns_for_pairs(bq, fx, start_date, end_date, freq)
    get_dendo_from_returns(returns, output,orientation)
    gui_fx.children = list(gui_fx.children[:-1])+[gui_output]


In [7]:
def compute_eq_dendo(bq, index, start_date, end_date, freq, ccy, output,orientation):
    gui_output = gui_eq.children[-1]
    gui_eq.children = list(gui_eq.children[:-1])+[spinner]
    returns = get_returns_for_eq_index_members(bq, index, start_date, end_date, freq, ccy)
    get_dendo_from_returns(returns, output,orientation)
    gui_eq.children = list(gui_eq.children[:-1])+[gui_output]


In [8]:
def compute_custom_dendo(bq, lst, start_date, end_date, freq, ccy, output,orientation):
    lst = lst.split('\n')
    lst = list(filter(None,lst))
    
    gui_output = gui_custom.children[-1]
    gui_custom.children = list(gui_custom.children[:-1])+[spinner]
    returns = get_returns_for_custom_list(bq, lst, start_date, end_date, freq, ccy)
    get_dendo_from_returns(returns, output,orientation)
    gui_custom.children = list(gui_custom.children[:-1])+[gui_output]


In [9]:
g10 = [f + 'USD Curncy' for f in ['EUR', 'JPY', 'GBP', 'CHF', 'CAD', 'AUD', 'NZD', 'NOK', 'SEK', 'DKK']]
dev = g10 + [f + 'USD Curncy' for f in ['HKD', 'ILS', 'SGD']]
main_em = [f + 'USD Curncy' for f in ['BRL', 'CLP', 'CNY', 'HKD','HUF','IDR','INR', 'KRW', 'MXN', 'MYR', 'PHP', 'PLN', 'RON', 'RUB', 'SGD', 'THB', 'TRY', 'TWD', 'ZAR']]
em = [f + 'USD Curncy' for f in ['ARS', 'BGN', 'BRL', 'CLP', 'CNY', 'COP', 'CZK', 'HKD','HUF','IDR','INR', 'KRW', 'MXN', 'MYR', 'PEN', 'PHP', 'PLN', 'RON', 'RUB', 'SGD', 'THB', 'TRY', 'TWD', 'ZAR']]
comm = ['XAU Curncy', 'XAG Curncy'] + [c + ' Comdty' for c in ['CO1', 'CL1', 'HG1', 'LA1', 'KC1']]

options = OrderedDict([
    ('G-10', g10),
#     ('Developed markets', dev),
    ('Developed markets and main EM', dev + main_em),
#     ('Developed and Emerging markets', dev + em),
#     ('Dev + Em + commodities', dev + em + comm),
])

In [10]:
# GUI

# title
title = AppTitle('Hierarchical Clustering', description='Analyse financial assets using Hierarchical Clustering with Bloomberg BQuant')
# inputs
univ_fx_ipy = DropdownAndLabel(options=options, label='FX Groups', width=200)
univ_eq_ipy = TextBoxAndLabel(label='Equity Index', width=200, init_value='SX5E Index')
sd_ipy = DatePickerAndLabel(label='Start Date', width=100, init_value='1999-12-31')
ed_ipy = DatePickerAndLabel(label='End Date', width=100, init_value='2019-12-31')
freq_ipy = DropdownAndLabel(options=['D', 'W', 'M', 'Q'], label='Frequency', width=100)
curr_ipy = DropdownAndLabel(options=['EUR', 'USD'], label='Currency', width=100)
orientation = DropdownAndLabel(options=['Horizontal', 'Vertical'],label='Orientation',width=100)
custom_list_box = Textarea(layout=Layout(width='160px', height='250px'), 
                                              placeholder='Please separate each ticker with a new line or drag and drop from excel.')
spinner = HTML('''<i class="fa fa-spinner fa-spin" style="font-size:24px"></i>''')


inputs_fx_ipy = HBox([
    univ_fx_ipy,
    orientation,
    sd_ipy,
    ed_ipy,
    freq_ipy,
])
inputs_eq_ipy = HBox([
    univ_eq_ipy,
    orientation,
    sd_ipy,
    ed_ipy,
    freq_ipy,
    curr_ipy,
])

custom_ipy = HBox([
    custom_list_box,
    orientation,
    sd_ipy,
    ed_ipy,
    freq_ipy,
    curr_ipy,
])

# outputs
out_fx_ipy = Output()
out_eq_ipy = Output()
out_custom_ipy = Output()
# buttons
btn_fx_ipy = ComputeButton(width=200, label='Compute FX Dendogram')
btn_fx_ipy.on_click(
        lambda x: compute_fx_dendo(bq,
                fx=univ_fx_ipy.value,
                start_date=sd_ipy.value,
                end_date=ed_ipy.value,
                freq=freq_ipy.value,
                output=out_fx_ipy,
                orientation=orientation.value,
                                
        )
)
btn_eq_ipy = ComputeButton(width=200, label='Compute Equity Dendogram')
btn_eq_ipy.on_click(
        lambda x: compute_eq_dendo(bq,
                index=univ_eq_ipy.value,
                start_date=sd_ipy.value,
                end_date=ed_ipy.value,
                freq=freq_ipy.value,
                ccy=curr_ipy.value,
                output=out_eq_ipy,
                orientation=orientation.value,                
                                
        )
)
btn_custom_ipy = ComputeButton(width=200, label='Compute custom list Dendogram')
btn_custom_ipy.on_click(
        lambda x: compute_custom_dendo(bq,
                lst=custom_list_box.value,
                start_date=sd_ipy.value,
                end_date=ed_ipy.value,
                freq=freq_ipy.value,
                ccy=curr_ipy.value,
                output=out_custom_ipy,
                orientation=orientation.value,                
                                
        )
)
# gui
gui_fx = VBox([
    inputs_fx_ipy,
    btn_fx_ipy,
    out_fx_ipy,
])
gui_eq = VBox([
    inputs_eq_ipy,
    btn_eq_ipy,
    out_eq_ipy,
])

gui_custom = VBox([
    custom_ipy,
    btn_custom_ipy,
    out_custom_ipy,
])

tabs = Tab([gui_fx, gui_eq,gui_custom])
tabs.set_title(0, 'FX')
tabs.set_title(1, 'Equity')
tabs.set_title(2, 'Custom list')
gui = VBox([title, tabs])
gui